# Training notebook

This notebook contains code to train the extended network using a train and validation set to obtain optimal hyperparameters.

In [1]:
import os

import numpy as np
import pandas as pd
import torch
import torch.optim as optim
import torch.nn as nn
import matplotlib.pyplot as plt
from statistics import mean
import matplotlib
from tqdm import tqdm
from datetime import datetime

import os
from PIL import Image
from sklearn.metrics import accuracy_score
import torchvision
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
from torch.utils.data import Dataset, DataLoader, ConcatDataset, SubsetRandomSampler
from torch.optim import lr_scheduler

plt.style.use('seaborn')

import DiagnosisFunctions.tools as tools

import torchvision.models as models

import albumentations as A
import torchvision.transforms.functional as TF
from sklearn.model_selection import KFold
import time
import pickle

import CNNmodels as CNNmodels

In [2]:
#Set the notebook to run on the GPU, if available.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f'This notebook is running on the {device.type}.')

if device.type == 'cuda':
    print(f"Running on device {torch.cuda.current_device()}")
    print('')

This notebook is running on the cpu.


In [3]:
(train_path, train_target), (test_path, test_target) = tools.get_splits_characteristics()

train_transform = A.Compose(
    [
        #ElasticTransform(alpha=1, sigma=50, alpha_affine=50, interpolation=1, border_mode=4, p=0.5),
        A.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.4, hue=0, always_apply=False, p=0.5),
        A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
    ]
)

train_set = tools.CharacteristicsDataset(path = train_path, target = train_target, size = [200, 200], transform = train_transform)
test_set = tools.CharacteristicsDataset(path = test_path, target = test_target, size = [200, 200])

In [4]:
image, target, characteristics = train_set[0]

In [5]:
def train_and_eval(phase, model, optimizer, criterion, scheduler, dataloaders):
    if phase == 'train':
        model.train()
    else:
        model.eval()

    running_loss = 0.0

    #Preallocate the probabilities dataframe.
    probabilities = pd.DataFrame(columns = dataloaders[phase].dataset.variables)
    ground_truth  = pd.DataFrame(columns = dataloaders[phase].dataset.variables)

    for inputs, targets, _ in dataloaders[phase]:
        inputs  = inputs.to(device)
        targets = targets.to(device).float()

        optimizer.zero_grad()

        with torch.set_grad_enabled(phase == 'train'):
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            if phase == 'train':
                loss.backward()
                optimizer.step()

            running_loss += loss.item()

        #Append to the dataframes
        probabilities = probabilities.append(pd.DataFrame(outputs.detach().cpu().numpy(), columns = dataloaders[phase].dataset.variables), ignore_index=True)
        ground_truth  = ground_truth.append(pd.DataFrame(targets.detach().cpu().numpy(), columns  = dataloaders[phase].dataset.variables), ignore_index=True)

    if phase == 'train':
        scheduler.step()

    #Return the total loss.
    return running_loss, ground_truth, probabilities

# Training

In [6]:
def score_predictions(gt, p):
    assert np.all(p.columns == gt.columns), 'Columns should be the same.'

    #Calculate the diagnosis f1 score.
    diagnosis_p = p[[x for x in p.columns if 'diagnosis_' in x]]
    diagnosis_gt = gt[[x for x in gt.columns if 'diagnosis_' in x]]
    assert np.all(diagnosis_p.columns == diagnosis_gt.columns), 'Columns should be the same'

    #Find the diagnosis f1 macro.
    diagnosis_p_pred  = diagnosis_p.values.argmax(axis=1)
    diagnosis_gt_pred = diagnosis_gt.values.argmax(axis=1) 
    diagnosis_f1      = f1_score(diagnosis_gt_pred, diagnosis_p_pred, average='macro')

    return diagnosis_f1

In [7]:
k = 5
num_epochs = 20

In [8]:
class WeightedBCELoss():
    def __init__(self, weights=[1, 1, 1]):
        self.weights = weights
        self.criterion = nn.BCELoss()

    def __call__(self, probabilities, targets):
        loss_characteristics = self.criterion(probabilities[:, :7], targets[:, :7]) 
        loss_diagnosis       = self.criterion(probabilities[:, 7:13], targets[:, 7:13]) 
        loss_area            = self.criterion(probabilities[:, 13:], targets[:, 13:])

        return self.weights[0] * loss_characteristics + self.weights[1] * loss_diagnosis + self.weights[2] * loss_area

In [13]:
def objective(trial):
    # parameters
    lr = trial.suggest_float("lr", 1e-5, 1e-3, log=True)
    batch_size = trial.suggest_int("batch_size", 32, 128, step=8)
    weights = [trial.suggest_float(f"weight_{type}", 0, 1) for type in ['characteristics', 'diagnosis', 'area']]

    # training code
    splits = KFold(n_splits=k)

    loss = {'train': [[] for _ in range(k)], 'val': [[] for _ in range(k)]}
    f1_characteristics = {'train': [[] for _ in range(k)], 'val': [[] for _ in range(k)]}
    f1_diagnosis = {'train': [[] for _ in range(k)], 'val': [[] for _ in range(k)]}
    f1_area = {'train': [[] for _ in range(k)], 'val': [[] for _ in range(k)]}

    for fold, (train_idx, val_idx) in enumerate(splits.split(np.arange(len(train_set)))):
        # Define train sampler and val sampler.
        train_sampler = SubsetRandomSampler(train_idx)
        val_sampler   = SubsetRandomSampler(val_idx)
        
        train_loader  = DataLoader(train_set, batch_size=batch_size, sampler=train_sampler)
        val_loader    = DataLoader(train_set, batch_size=batch_size, sampler=val_sampler)

        cnn = CNNmodels.CNN(n_characteristics=7, n_diagnosis=6, n_area=4).to(device)
        
        criterion = WeightedBCELoss(weights=weights)
        optimizer = optim.Adam(cnn.parameters(), lr=lr, weight_decay=1e-4)
        scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

        #Update the dataloaders passed to the training function.
        dataloaders = {'train' : train_loader, 'val' : val_loader}

        for epoch in tqdm(range(num_epochs), desc=f'Fold {fold}', unit='epoch'):
            for phase in ['train', 'val']:
                epoch_loss, gt, p = train_and_eval(phase, cnn, optimizer, criterion, scheduler, dataloaders)

                if phase == 'train':
                    avg_obs_loss = (epoch_loss / len(train_idx)) 
                elif phase == 'val':
                    avg_obs_loss = (epoch_loss / len(val_idx))

                loss[phase][fold].append(avg_obs_loss)

                # Predict labels based on probabilities
                pred_class = tools.classify_probability_predictions(p.copy())
                
                # Compute f1 scores with average 'samples' (default values)
                metric_dict = tools.compute_metrics_scores(gt, pred_class)
                
                f1_characteristics[phase][fold].append(metric_dict['characteristics'])
                f1_diagnosis[phase][fold].append(metric_dict['diagnosis'])
                f1_area[phase][fold].append(metric_dict['area'])

    #Save the results to a pickle.
    with open(f'results/CharacteristicStats_{datetime.now().__str__()}.p', 'wb') as output_file:
        pickle.dump([num_epochs, k, loss, (f1_diagnosis, f1_characteristics, f1_area)], output_file)

    return np.mean(f1_diagnosis['val'])

In [ ]:
import optuna

print("Starting Optuna study")
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

with open(f"studies/{datetime.now().__str__()}.p", 'wb') as output_file:
    pickle.dump(study, output_file)